### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

### Importing Datasets

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_df =pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
train_df.head()

### Splitting Date Column

In [ ]:
train_df['year'] = pd.to_datetime(train_df['date']).dt.year
train_df['week'] = pd.to_datetime(train_df['date']).dt.week
train_df['day'] = pd.to_datetime(train_df['date']).dt.day
train_df['weekday'] = pd.to_datetime(train_df['date']).dt.dayofweek

test_df['year'] = pd.to_datetime(test_df['date']).dt.year
test_df['week'] = pd.to_datetime(test_df['date']).dt.week
test_df['day'] = pd.to_datetime(test_df['date']).dt.day
test_df['weekday'] = pd.to_datetime(test_df['date']).dt.dayofweek

train_df.drop('date', axis=1, inplace=True)
test_df.drop('date', axis=1, inplace=True)
train_df.head()

### Converting Categorical Features into Numerical By One Hot Encoding

In [ ]:
train_ohe = pd.get_dummies(train_df, columns = ['country', 'store', 'product'])
test_ohe = pd.get_dummies(test_df, columns = ['country', 'store', 'product'])
train_ohe.head()

### Droping Irrelevent Features 

In [ ]:
train_ohe.drop('row_id', axis=1, inplace=True)
test_ohe.drop('row_id', axis=1, inplace=True)
train_ohe.head()

In [ ]:
train_ohe.drop('num_sold', axis=1, inplace=True)

In [ ]:
print('trian shape',train_ohe.shape)
print('test shape',test_ohe.shape)

### Creating Two-Way Interaction between all Features

In [ ]:
# Use PolynomialFeatures in sklearn.preprocesingto create two-way interaction for all features
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):

    # Get feature name
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]

    # Find Interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames

    # Removing INteractions terms with all 0 values
    noint_indices = [i for i, x in enumerate(list((df==0).all())) if x]
    df = df.drop(df.columns[noint_indices], axis=1)

    return df

In [ ]:
X = add_interactions(train_ohe)
test_ohe = add_interactions(test_ohe)
X.head()

In [ ]:
print('X shape: ',X.shape)

### Conclusion:
<b>*It added many extra features to dataset
<br>71 Features can lead to curse of dimension
<br>So need to reduce the Dimension using PCA*</b>

### Dimensionality Reduction Using PCA

In [ ]:
# Use PCA from sklearn.decomposition to find principle component
from sklearn.decomposition import PCA

pca = PCA(n_components=15)
X_pca = pd.DataFrame(pca.fit_transform(X))
test_ohe_pca = pd.DataFrame(pca.fit_transform(test_ohe ))
X_pca.head()